In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib.request
import time

In [ ]:
authors = []
dates = []
statements = []
sources = []
targets = []

In [ ]:
#we cerated a function to scrape a website
def scrape_website(page_number):
  page_num = str(page_number)
  URL = 'https://www.politifact.com/factchecks/list/?page='+page_num
  webpage = requests.get(URL)
  soup = BeautifulSoup(webpage.text,'html.parser')
  
  #location 
  statement_footer = soup.find_all('footer', attrs={'class':'m-statement__footer' })# location of author  
  statement_quote = soup.find_all('div', attrs={'class':'m-statement__quote' }) #location of the statement
  statement_meta = soup.find_all('div', attrs={'class':'m-statement__meta' }) #location of the source
  target = soup.find_all('div', attrs={'class':'m-statement__meter' }) # location of the target (score card)



  #made a loop for the statement_footer
  for i in statement_footer:
    link1 = i.text.strip()
    name_and_date = link1.split()
    first_name = name_and_date[1]
    last_name = name_and_date[2]
    full_name = first_name +' ' + last_name
    month = name_and_date[4]
    day = name_and_date[5]
    year = name_and_date[6]
    date = month +' ' + day + ' ' + year
    dates.append(date)
    authors.append(full_name) 


    #we made a loop for the statement_quote
  for i in statement_quote:
    link2 = i.find_all('a')
    statement_text = link2[0].text.strip()
    statements.append(statement_text)


  #we created a loop for the meta
  for i in statement_meta:
    link3 = i.find_all('a')
    source_text = link3[0].text.strip()
    sources.append(source_text)

  #loop for target
  for i in target:
    link4 = i.find('div', attrs = {'class' , 'c-image'}).find('img').get('alt')  
    targets.append(link4) 
  

In [ ]:
#loop for 'n-1' webpages to scrape the data
n = 96

for i in range(1,n):
  scrape_website(i)


In [ ]:
#create the dataframe
data= pd.DataFrame(columns = ['author','statement','source','date', 'target'])
data['author']= authors
data['statement']= statements
data['source'] = sources
data['date'] = dates
data['target'] = targets
data

,author,statement,source,date,target
0,Samantha Putterman,Ukrainians altered this street sign to tell Ru...,Facebook posts,"March 2, 2022",false
1,Paul Specht,"There are now zero students, staff or anyone “...",David Willis,"March 2, 2022",half-true
2,Madison Czopek,Photos show Ukrainian citizens destroying two ...,Facebook posts,"March 2, 2022",false
3,Louis Jacobson,“Our economy created over 6.5 million new jobs...,Joe Biden,"March 1, 2022",half-true
4,Meg Doster,"""According to Republican Rep. Jeff Shipley, te...",Iowa House Democrats,"March 1, 2022",mostly-true
...,...,...,...,...,...
2845,Eric Litke,Says a picture of stacked mailboxes is proof o...,Facebook posts,"August 17, 2020",false
2846,Victoria Knight,“[Steve] Bullock’s health care plan will force...,National Republican Senatorial Committee,"August 17, 2020",false
2847,Ciara O'Rourke,Photo shows Barack Obama and George Clooney in...,Viral image,"August 17, 2020",false
2848,Jill Terreri,“New York Mail-In voting is in a disastrous st...,Donald Trump,"• August 14,",barely-true


In [ ]:
print(len(authors))

2850


In [ ]:
#created a function to get a binary number
def getBinaryNumTarget(text):
  if text == 'true':
    return 1
  else:
    return 0


In [ ]:
#created a function to get a label 
def getBinaryTarget(text):
  if text == 'true':
    return 'Real'
  else:
    return 'Fake'

In [ ]:
data['Binary'] = data['target'].apply(getBinaryNumTarget)
data['Label'] = data['target'].apply(getBinaryTarget)
data

,author,statement,source,date,target,Binary,Label
0,Samantha Putterman,Ukrainians altered this street sign to tell Ru...,Facebook posts,"March 2, 2022",false,0,Fake
1,Paul Specht,"There are now zero students, staff or anyone “...",David Willis,"March 2, 2022",half-true,0,Fake
2,Madison Czopek,Photos show Ukrainian citizens destroying two ...,Facebook posts,"March 2, 2022",false,0,Fake
3,Louis Jacobson,“Our economy created over 6.5 million new jobs...,Joe Biden,"March 1, 2022",half-true,0,Fake
4,Meg Doster,"""According to Republican Rep. Jeff Shipley, te...",Iowa House Democrats,"March 1, 2022",mostly-true,0,Fake
...,...,...,...,...,...,...,...
2845,Eric Litke,Says a picture of stacked mailboxes is proof o...,Facebook posts,"August 17, 2020",false,0,Fake
2846,Victoria Knight,“[Steve] Bullock’s health care plan will force...,National Republican Senatorial Committee,"August 17, 2020",false,0,Fake
2847,Ciara O'Rourke,Photo shows Barack Obama and George Clooney in...,Viral image,"August 17, 2020",false,0,Fake
2848,Jill Terreri,“New York Mail-In voting is in a disastrous st...,Donald Trump,"• August 14,",barely-true,0,Fake


In [ ]:
data.Binary.value_counts()

0    2747
1     103
Name: Binary, dtype: int64

In [ ]:
#data.to_csv('Politifact.csv')